# RCOE GenAI Agents: Colab Deployment with Ngrok

This notebook sets up and deploys the RCOEGenAIAgents Flask application in Google Colab with ngrok public URL:
- **RCOEGenAIAgents.py** (port 5000) - MCP Architecture with Pure Gen AI Intent Routing

It clones the repo to fetch required files automatically, and handles secrets via upload of `.env` and `oci_api_key.pem`.

Security notes:
- Do not print secrets in output cells.
- Upload `.env` and `oci_api_key.pem` at runtime; do not embed secrets in the notebook.
- ngrok token is hardcoded here per request; rotate it when finished.

In [ ]:
# 1) Install Required Dependencies
import sys
import subprocess

packages = [
    'flask', 'python-dotenv', 'requests', 'oci', 'pyngrok==7.1.2'
]

for pkg in packages:
    print(f"Installing {pkg}...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print("All packages installed.")

In [ ]:
# 2) Import Libraries and Configure Environment
import os
import time
import json
import threading
import textwrap
import shutil
import signal
from pathlib import Path
from dotenv import load_dotenv
from pyngrok import ngrok, conf

WORKDIR = Path('/content/OCIGenAIBot')
WORKDIR.mkdir(parents=True, exist_ok=True)
print(f"Working directory: {WORKDIR}")

# Ensure ngrok default config doesn't reuse old tunnels in Colab sessions
conf.get_default().monitor_thread = False

## Option A: Clone from GitHub (if repo is public)

If your repository is public and contains all required files, you can clone it instead of uploading files manually. **Note:** Do NOT commit `.env` or `oci_api_key.pem` to GitHub for security reasons. You'll still need to upload those files separately.

In [ ]:
# Option A: Clone GitHub Repository (Enabled by default)
GITHUB_REPO = "https://github.com/Ramsiit2010/OCIGenAIBot.git"

import subprocess
import shutil
from pathlib import Path

# Fresh clone directory
CLONE_DIR = Path('/content/OCIGenAIBot_clone')
if CLONE_DIR.exists():
    shutil.rmtree(CLONE_DIR)

subprocess.check_call(['git', 'clone', GITHUB_REPO, str(CLONE_DIR)])
print(f"Cloned {GITHUB_REPO}")

# Copy required files into WORKDIR
files_to_copy = [
    'RCOEGenAIAgents.py',
    'config.properties',
    'mcp_servers/__init__.py',
    'mcp_servers/base_server.py',
    'mcp_servers/advisors.py',
    'api_spec_general.json',
    'api_spec_finance.json',
    'api_spec_hr.json',
    'api_spec_orders.json',
    'api_spec_reports.json'
]

# Create target directories
(WORKDIR / 'mcp_servers').mkdir(exist_ok=True)

for file_path in files_to_copy:
    src = CLONE_DIR / file_path
    if src.exists():
        dest = WORKDIR / file_path
        dest.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dest)
        print(f"Copied: {file_path}
")
    else:
        print(f"Warning: {file_path} not found in repo")

print("\nIMPORTANT: Upload .env and oci_api_key.pem in the next cell.")


In [ ]:
# Option A (continued): Upload Secret Files (.env and oci_api_key.pem)
# Run this cell AFTER running Option A to upload sensitive files

from google.colab import files

print("Upload ONLY secret files (do NOT commit these to GitHub):")
print("- .env")
print("- oci_api_key.pem")
print()

secret_uploads = files.upload()

for name, data in secret_uploads.items():
    with open(WORKDIR / name, 'wb') as f:
        f.write(data)
    print(f"Saved: {name}")

# Set PEM permissions
pem_path = WORKDIR / 'oci_api_key.pem'
if pem_path.exists():
    os.chmod(pem_path, 0o600)
    print("✓ PEM permissions set to 600")

# Load .env
load_dotenv(dotenv_path=WORKDIR / '.env')
print("✓ .env loaded")

print("\n✓ Secret files uploaded. Ready to proceed.")

## Notes on Configuration and Security

- RCOEGenAIAgents.py uses **MCP (Model Context Protocol)** architecture with 5 specialized advisors
- The application reads `genai_region` from `config.properties` and uses the local `oci_api_key.pem` from the working folder
- `.env` must contain: `OCI_USER`, `OCI_FINGERPRINT`, `OCI_TENANCY`, `OCI_KEY_FILE=oci_api_key.pem`
- Do not display or print secret values in the notebook output

In [ ]:
# 3) Set Up Ngrok (Auth Token) and Create Tunnel
from pyngrok import ngrok

# Hardcoded token as requested (note: embedding secrets is generally insecure)
NGROK_AUTH_TOKEN = "35KIlIDMzheum8MLvD4kqv39A4q_RD1X1u3CM4MhztuXkadu"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Ensure any previous tunnels are closed
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except Exception:
        pass

print("Starting ngrok tunnel for port 5000...")
http_tunnel_5000 = ngrok.connect(5000, bind_tls=True)

print("\n✓ Public URL:")
print("RCOEGenAIAgents:", http_tunnel_5000.public_url)

In [ ]:
# 4) Launch RCOEGenAIAgents Flask Application
import subprocess
import sys
from pathlib import Path

process = None

os.chdir(WORKDIR)

# Start the application
script = 'RCOEGenAIAgents.py'
if not Path(script).exists():
    raise FileNotFoundError(f"{script} not found in {WORKDIR}")

env = os.environ.copy()
env['PYTHONUNBUFFERED'] = '1'
process = subprocess.Popen([sys.executable, script], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print(f"✓ Started RCOEGenAIAgents on port 5000 (PID={process.pid})")
print(f"✓ Access at: {http_tunnel_5000.public_url}")

In [ ]:
# 5) Stream Application Logs (Optional)
print("Streaming application logs for ~60 seconds. Stop the cell to end early.")
print("=" * 60)
start_time = time.time()

while time.time() - start_time < 60:
    if process.poll() is not None:
        print(f"\n⚠️ Process exited with code {process.returncode}")
        break
    
    # Non-blocking read
    line = process.stdout.readline()
    if line:
        print(line, end='')
    time.sleep(0.1)

print("\n" + "=" * 60)
print("Log tailing complete. Use the public URL above to access the app.")

In [ ]:
# 6) Health Check: Verify MCP Servers Registration
try:
    print("RCOEGenAIAgents URL:", http_tunnel_5000.public_url)
    print()
    
    # Check MCP servers registration
    import requests
    url_servers = http_tunnel_5000.public_url + '/mcp/servers'
    print(f"Checking MCP server registration at: {url_servers}")
    
    r = requests.get(url_servers, timeout=10)
    print(f"Status: {r.status_code}")
    
    if r.ok:
        data = r.json()
        print(f"\n✓ Total MCP Servers: {data.get('total', 0)}")
        print(f"✓ Gen AI Enabled: {data.get('genai_enabled', False)}")
        print(f"✓ Gen AI Region: {data.get('genai_region', 'N/A')}")
        print("\nRegistered Servers:")
        for server in data.get('servers', []):
            print(f"  - {server.get('name')}: {server.get('status')}")
    else:
        print(f"⚠️ Health check failed: {r.status_code}")
        
except Exception as e:
    print(f"⚠️ Health check error: {e}")

In [ ]:
# 7) Graceful Shutdown Helper

def stop_all():
    """Stop the application and close ngrok tunnel"""
    print("Stopping tunnel and process...")
    
    # Close ngrok tunnel
    try:
        ngrok.disconnect(http_tunnel_5000.public_url)
        ngrok.kill()
        print("✓ Ngrok tunnel closed.")
    except Exception as e:
        print(f"⚠️ Ngrok shutdown error: {e}")
    
    # Terminate Flask process
    if process:
        try:
            process.terminate()
            try:
                process.wait(timeout=5)
            except subprocess.TimeoutExpired:
                process.kill()
            print(f"✓ Terminated RCOEGenAIAgents (PID={process.pid})")
        except Exception as e:
            print(f"⚠️ Error stopping process: {e}")
    
    print("\n✓ Shutdown complete.")

print("Run stop_all() in a new cell to shut everything down when finished.")